In [15]:
import pandas as pd
import requests
from zipfile import ZipFile
import io
from checkpoints import checkpoints
from pandas.tseries.offsets import MonthBegin

In [7]:
data_ranges = pd.date_range(start='2013-07-01', end='2017-01-01', freq='M').map(lambda dt: (dt.year, dt.month))

In [12]:
uris = ["https://s3.amazonaws.com/tripdata/{0}{1}-citibike-tripdata.zip".format(y, str(d).zfill(2)) for (y, d) in data_ranges]

In [16]:
checkpoints.enable()

In [115]:
def datafy(uri):
    print(uri)
    
    # Get the month start and end.
    ym = uri.split("/")[-1][:6]
    y, m = ym[:4], ym[4:].lstrip("0")
    
    tstart = pd.to_datetime("{0}-{1}".format(y, m))
    tend = tstart + MonthBegin(1)
    
    print(tstart, tend)
    
    # Make the request.
    print("Requesting data...")
    r = requests.get(uri, timeout=5)
    print("Request complete. Munging data...")
    
    # Unzip the contents.
    myzip = ZipFile(io.BytesIO(r.content))
    with myzip.open(myzip.filelist[0].filename) as fp:
        data = pd.read_csv(fp)
        
    # Get the start/end time columns---which name is used depends on which dataset.
    # Used to be "starttime"/"endtime" columns, now "Start Time"/"End Time" columns.
    start_time_col = 'Start Time' if 'Start Time' in data.columns else 'starttime'
    stop_time_col = 'Stop Time' if 'Stop Time' in data.columns else 'stoptime'
    
    # Ditto with some other columns.
    trip_duration_col = 'Trip Duration' if 'Trip Duration' in data.columns else 'tripduration'
    start_station_name_col = 'Start Station Name' if 'Start Station Name' in data.columns else 'start station name'
    end_station_name_col = 'End Station Name' if 'End Station Name' in data.columns else 'end station name'
    start_station_latitude_col = 'Start Station Latitude' if 'Start Station Latitude' in data.columns else 'start station latitude'
    start_station_longitude_col = 'Start Station Longitude' if 'Start Station Longitude' in data.columns else 'start station longitude'
    start_station_id_col = 'Start Station ID' if 'Start Station ID' in data.columns else 'start station id'
    
    # Aggregate start times.
    data[start_time_col] = pd.to_datetime(data[start_time_col])
    accum_start = (data
        .set_index(start_time_col)
        .groupby([pd.TimeGrouper(freq='H'), start_station_name_col])
        .count()
        .reset_index())
    accum_start = accum_start[[start_time_col, start_station_name_col, trip_duration_col]]\
        .rename(columns={trip_duration_col: 'Outbounds'})
    accum_totals = accum_start.groupby(start_station_name_col).apply(
        lambda df: df.set_index(start_time_col)\
            .reindex(
                pd.DatetimeIndex(freq='H', start=tstart, end=tend),
                fill_value=0
            )\
            .drop(start_station_name_col, axis='columns')\
            .reset_index()
    ).reset_index().drop('level_1', axis='columns').rename(columns={'index': 'Time'})
    
    # Mix in identifying information.
    identifying_information = data.set_index(start_station_name_col)[
        [start_station_latitude_col, start_station_longitude_col, start_station_id_col]
    ].drop_duplicates()
    accum_totals = accum_totals.join(identifying_information, on=start_station_name_col)
    
    # Add ending information
    data[stop_time_col] = pd.to_datetime(data[stop_time_col])
    accum_end = (data
        .set_index(stop_time_col)
        .groupby([pd.TimeGrouper(freq='H'), end_station_name_col])
        .count()
        .reset_index())
    accum_end = accum_end[[stop_time_col, end_station_name_col, trip_duration_col]]\
        .rename(columns={trip_duration_col: 'Inbounds'})
    accum_end = accum_end.groupby(end_station_name_col).apply(
        lambda df: df.set_index(stop_time_col)\
            .reindex(
                pd.DatetimeIndex(freq='H', start=tstart, end=tend),
                fill_value=0
            )\
            .drop(end_station_name_col, axis='columns')\
            .reset_index()
    ).reset_index().drop('level_1', axis='columns').rename(columns={'index': 'Time'})
    
    # Merge.
    accum_totals = pd.merge(accum_totals, accum_end, how='outer', 
                            left_on=[start_station_name_col, 'Time'], right_on=[end_station_name_col, 'Time'])
    
    # Clean up.
    accum_totals = accum_totals.dropna()
    accum_totals['Outbounds'] = accum_totals['Outbounds'].astype(int)
    accum_totals[start_station_id_col] = accum_totals[start_station_id_col].astype(int)
    
    return accum_totals

In [83]:
datum = pd.Series(uris).safe_map(datafy)

2013-07-01 00:00:00 2013-08-01 00:00:00
2013-08-01 00:00:00 2013-09-01 00:00:00
2013-09-01 00:00:00 2013-10-01 00:00:00
2013-01-01 00:00:00 2013-02-01 00:00:00
2013-11-01 00:00:00 2013-12-01 00:00:00
2013-12-01 00:00:00 2014-01-01 00:00:00
2014-01-01 00:00:00 2014-02-01 00:00:00
2014-02-01 00:00:00 2014-03-01 00:00:00
2014-03-01 00:00:00 2014-04-01 00:00:00
2014-04-01 00:00:00 2014-05-01 00:00:00
2014-05-01 00:00:00 2014-06-01 00:00:00
2014-06-01 00:00:00 2014-07-01 00:00:00
2014-07-01 00:00:00 2014-08-01 00:00:00
2014-08-01 00:00:00 2014-09-01 00:00:00
2014-09-01 00:00:00 2014-10-01 00:00:00
2014-01-01 00:00:00 2014-02-01 00:00:00


KeyboardInterrupt: 

In [116]:
datum = pd.Series(uris).safe_map(datafy)

https://s3.amazonaws.com/tripdata/201410-citibike-tripdata.zip
2014-10-01 00:00:00 2014-11-01 00:00:00
Requesting data...
Request complete. Munging data...
https://s3.amazonaws.com/tripdata/201411-citibike-tripdata.zip
2014-11-01 00:00:00 2014-12-01 00:00:00
Requesting data...
Request complete. Munging data...
https://s3.amazonaws.com/tripdata/201412-citibike-tripdata.zip
2014-12-01 00:00:00 2015-01-01 00:00:00
Requesting data...
Request complete. Munging data...
https://s3.amazonaws.com/tripdata/201501-citibike-tripdata.zip
2015-01-01 00:00:00 2015-02-01 00:00:00
Requesting data...
Request complete. Munging data...
https://s3.amazonaws.com/tripdata/201502-citibike-tripdata.zip
2015-02-01 00:00:00 2015-03-01 00:00:00
Requesting data...
Request complete. Munging data...
https://s3.amazonaws.com/tripdata/201503-citibike-tripdata.zip
2015-03-01 00:00:00 2015-04-01 00:00:00
Requesting data...
Request complete. Munging data...
https://s3.amazonaws.com/tripdata/201504-citibike-tripdata.zip
2

KeyboardInterrupt: 

In [125]:
datum = pd.Series(uris).safe_map(datafy)

https://s3.amazonaws.com/tripdata/201508-citibike-tripdata.zip
2015-08-01 00:00:00 2015-09-01 00:00:00
Requesting data...
Request complete. Munging data...
https://s3.amazonaws.com/tripdata/201509-citibike-tripdata.zip
2015-09-01 00:00:00 2015-10-01 00:00:00
Requesting data...
Request complete. Munging data...
https://s3.amazonaws.com/tripdata/201510-citibike-tripdata.zip
2015-10-01 00:00:00 2015-11-01 00:00:00
Requesting data...
Request complete. Munging data...
https://s3.amazonaws.com/tripdata/201511-citibike-tripdata.zip
2015-11-01 00:00:00 2015-12-01 00:00:00
Requesting data...
Request complete. Munging data...
https://s3.amazonaws.com/tripdata/201512-citibike-tripdata.zip
2015-12-01 00:00:00 2016-01-01 00:00:00
Requesting data...
Request complete. Munging data...
https://s3.amazonaws.com/tripdata/201601-citibike-tripdata.zip
2016-01-01 00:00:00 2016-02-01 00:00:00
Requesting data...
Request complete. Munging data...
https://s3.amazonaws.com/tripdata/201602-citibike-tripdata.zip
2

In [140]:
data = pd.concat(datum.map(lambda df: (df
                      .rename(columns={n: n.title() for n in datum.iloc[0].columns})
                      .rename(columns={'Start Station Id': 'Start Station ID'}))).values)

In [142]:
data = data.dropna()

In [145]:
data['Inbounds'] = data['Inbounds'].astype(int)

In [146]:
data

,Start Station Name,Time,Outbounds,Start Station Latitude,Start Station Longitude,Start Station ID,End Station Name,Inbounds
0,1 Ave & E 15 St,2013-07-01 00:00:00,2,40.732219,-73.981656,504,1 Ave & E 15 St,1
1,1 Ave & E 15 St,2013-07-01 01:00:00,1,40.732219,-73.981656,504,1 Ave & E 15 St,2
2,1 Ave & E 15 St,2013-07-01 02:00:00,0,40.732219,-73.981656,504,1 Ave & E 15 St,1
3,1 Ave & E 15 St,2013-07-01 03:00:00,0,40.732219,-73.981656,504,1 Ave & E 15 St,0
4,1 Ave & E 15 St,2013-07-01 04:00:00,0,40.732219,-73.981656,504,1 Ave & E 15 St,0
5,1 Ave & E 15 St,2013-07-01 05:00:00,0,40.732219,-73.981656,504,1 Ave & E 15 St,1
6,1 Ave & E 15 St,2013-07-01 06:00:00,2,40.732219,-73.981656,504,1 Ave & E 15 St,1
7,1 Ave & E 15 St,2013-07-01 07:00:00,4,40.732219,-73.981656,504,1 Ave & E 15 St,2
8,1 Ave & E 15 St,2013-07-01 08:00:00,3,40.732219,-73.981656,504,1 Ave & E 15 St,3
9,1 Ave & E 15 St,2013-07-01 09:00:00,2,40.732219,-73.981656,504,1 Ave & E 15 St,2


In [149]:
data.to_csv("E:\citibike-hourly\data\citibike-hourly.csv", index=False)